# Wheel Loader GetValueRequest
This is a KWH HMI which allows to request the wheel loader's location. This Notebook acts as an **HMI** in terms of the S³I and a **client** in terms of OAuth authentication. Enter the id of your HMI with the corresponding secret as *hmi* in this script to make this notebook to your HMI. Running this notebook, you will authorize your HMI to call up the wheel loader's location. 
Just go to the **Cell** drop-down menu and use the **Run All** button.

First, all necessary modules are imported into the script, including the S³I library.

In [ ]:
import s3i
import json
import uuid
import jwt
import time
import os
import base64
import getpass
import requests
from tools import print_with_timestamp

In order to use the S³I this notebook needs a client id and the respective secret. You can assign this notebook to your personal HMI, to make this notebook your HMI. Therefore enter the id and the secret of your HMI in the following input fields.

In [ ]:
print_with_timestamp("Assign a client to this notebook. (The id of your HMI)")
hmiId = input('[S3I]: Please enter your HMI id:')
hmiSecret = getpass.getpass('[S3I]: Please enter your HMI secret:')
print_with_timestamp("HMI id and secret are set")

Next, you have to enter your username and password. With your access data a token is requested which authorizes this client (your HMI) to call up the wheel loader's location on your behalf.

In [ ]:
print_with_timestamp("DEMO Wheel loader, please log in!")
username = input('[S3I]: Please enter your username:')
password = getpass.getpass('[S3I]: Please enter the password:')
print_with_timestamp("Your credentials are sent to S3I IdentityProvider.")
s3i_identity_provider = s3i.IdentityProvider(grant_type='password', 
                                             identity_provider_url="https://idp.s3i.vswf.dev/",
                                             realm='KWH',
                                             client_id=hmiId,
                                             client_secret=hmiSecret,
                                             username=username,
                                             password=password)
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)

''' decode the access token
'''
parsed_username = jwt.decode(access_token, verify=False)[
    "preferred_username"]

print_with_timestamp("Token received, " + parsed_username + " logged in.")

After the token has been received, it can be used to query the endpoint of the wheel loader. To query the wheel loader in the S³I Directory, you must have the rights to view the wheel loader. With the creation of your S³I account you should have got this right. In case of problems please contact the S³I team (s3i@kwh40.de).

Besides the endpoint of the wheel loader, the enpoint of this HMI is also needed to add as "ReplyToEnpoint" field in the request. The endpoint of this HMI is also queried from the directory, as it may has changed. By querying its own endpoint from the directory, the HMI is independent of changes made to its own endpoint. These changes then only need to be listed in the directory.

In [ ]:
''' authentication with JWT in S3I Directory 
'''

s3i_directory = s3i.Directory(
    s3i_dir_url="https://dir.s3i.vswf.dev/api/2/", token=access_token)

print_with_timestamp("Authentication with Token in S3I Directory successful")
    
''' query the sender's endpoint
'''
sender_endpoint = s3i_directory.queryThingIDBased(hmiId+"/attributes/allEndpoints")[0]
    
''' query the wheel loaders's Id and endpoint
'''
wheelloaderID = s3i_directory.queryAttributeBased(
    "name", "wheelloader")[0]["thingId"]

wheelloaderEndpoint = s3i_directory.queryThingIDBased(wheelloaderID+"/attributes/allEndpoints")

receivers = [wheelloaderID]
receiver_endpoints = wheelloaderEndpoint

After querying the enpoints, the message is prepared. Since the location is one value of the wheel loader, a *GetValueRequest* is instantiated and filled in.

In [ ]:
print_with_timestamp("Prepare the GetValueRequest.")
attributePath = input(
        '[S3I]: please enter the attribute path of the value you want to receive from the wheel loader (e.g., attributes/features/ml40::Location OR attributes/features/ml40::Composite/targets/ml40::Tank/features/ml40::LiquidFillingLevel/currentLevel): \n')

msg_uuid = "s3i:" + str(uuid.uuid4())

servReq = s3i.GetValueRequest()
servReq.fillGetValueRequest(senderUUID=hmiId, receiverUUID=receivers, sender_endpoint=sender_endpoint,
                                attributePath=attributePath, msgUUID=msg_uuid)
print_with_timestamp("GetValueRequest prepared:")
print(json.dumps(servReq.msg, indent=2))

The HMI requests a new token to establish a connection to the broker and sends the request to the wheel loader's endpoint. It then checks incoming responses.

In [ ]:
print_with_timestamp("Sending the GetValueRequest to the wheel loader")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
#print(receiver_endpoints)
response = requests.post(url="https://broker.s3i.vswf.dev/"+receiver_endpoints[0] ,
                                data=json.dumps(servReq.msg), headers=headers)
print_with_timestamp(response.text)

In [ ]:
print_with_timestamp("Check the wheel loaders response")
access_token = s3i_identity_provider.get_token(s3i.TokenType.ACCESS_TOKEN)
headers = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + access_token}
response = requests.get(url="https://broker.s3i.vswf.dev/"+sender_endpoint , headers=headers)
json_acceptable_string = response.text.replace("'", "\"")

if json_acceptable_string:
    print(json.dumps(json_acceptable_string["value"], indent=2))
    #response_json = json.loads(json_acceptable_string)
    
    #print_with_timestamp("The wheel loader responded with:")
    #print(30*" "+"Longitude: ", response_json["value"]["longitude"])
    #print(30*" "+"Latitude: ", response_json["value"]["latitude"])
else:
    print_with_timestamp("The wheel did not respond yet.")